In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from ipywidgets import interact
import os

In [2]:
au_to_cm_1 = 219474.6305

wc = np.linspace(0, 1, 1000) #Original is 0, 1, 1000
wc = np.round(wc, decimals=3)

In [ ]:
#if you want to look at a specific reaction, load it here
#format the text file with three columns of perm dipoles, one for each direction x,y,z
#include as many coordinates as you want, but be sure to select the R, TS, P below
p_dip_x, p_dip_y, p_dip_z = np.loadtxt("reaction_permanent_dipoles.txt", comments=["#"], unpack=True)
pdR = p_dip_z[0]
pdT = p_dip_z[1]
pdP = p_dip_z[2]

In [ ]:
#specific reaction, this is designed for Orca DFT outputs
#load the correspoding absorption spectra for the R, TS, and P coordinates
#select the column of the direction of interest
specR = np.loadtxt("spectrum_reactant.txt")
tdR = specR[:, 7] #z direction
EnR = specR[:, 1]/au_to_cm_1

specT = np.loadtxt("spectrum_TS.txt")
tdT = specT[:, 7] #z direction
EnT = specT[:, 1]/au_to_cm_1

specP = np.loadtxt("spectrum_product.txt")
tdP = specP[:, 7] #z direction
EnP = specP[:, 1]/au_to_cm_1

In [5]:
def dR2(tdR, tr, pdR, pr):
    return sum(abs(tdR + tr)**2) + (pdR + pr)**2

def dT2(tdT, tt, pdT, pt):
    return sum(abs(tdT + tt)**2) + (pdT + pt)**2

def dP2(tdP, tp, pdP, pp):
    return sum(abs(tdP + tp)**2) + (pdP + pp)**2

In [6]:
def aRw(tdR, tr, EnR, er, w):
    pol = np.zeros(len(w))
    for i, x in enumerate(w):
        pol[i] = sum(abs(tdR + tr)**2/((EnR + er) + x))
    return pol

def aTw(tdT, tt, EnT, et, w):
    pol = np.zeros(len(w))
    for i, x in enumerate(w):
        pol[i] = sum(abs(tdT + tt)**2/((EnT + et) + x))
    return pol

def aPw(tdP, tp, EnP, ep, w):
    pol = np.zeros(len(w))
    for i, x in enumerate(w):
        pol[i] = sum(abs(tdP + tp)**2/((EnP + ep) + x))
    return pol

In [7]:
def m(tr, tt, tp, er, et, ep, pr, pt, pp):
    lfer = (dT2(tdT, tt, pdT, pt) - dR2(tdR, tr, pdR, pr) -  wc*(aTw(tdT, tt, EnT, et, wc) - aRw(tdR, tr, EnR, er, wc))) / (dP2(tdP, tp, pdP, pp) - dR2(tdR, tr, pdR, pr) - wc*(aPw(tdP, tp, EnP, ep, wc) - aRw(tdR, tr, EnR, er, wc)))
    fig, axes = plt.subplots(nrows=2, ncols=3, sharex='col', figsize=(13.5, 7))
    ax1, ax2, ax3, ax4, ax5, ax6 = axes.flatten()
    ax1.plot(wc, lfer)
    ax1.set_title(f'm over $\omega$')
    ax1.set_ylabel('m')
    ax1.set_xlabel('$\omega$')
    ax1.grid(True)
    ax1.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
    ax2.axhline(y=dR2(tdR, tr, pdR, pr), c="red", label="R")
    ax2.axhline(y=dT2(tdT, tt, pdT, pt), c="blue", label="TS")
    ax2.axhline(y=dP2(tdP, tp, pdP, pp), c="orange", label="P")
    ax2.grid(True)
    ax2.legend(loc='best', prop={'size': 8})
    ax2.set_ylabel("Dipole")
    ax2.set_xlabel('$\omega$')
    ax3.axhline(dT2(tdT, tt, pdT, pt) - dR2(tdR, tr, pdR, pr), c="purple", label ="TS-R")
    ax3.axhline(dP2(tdP, tp, pdP, pp) - dR2(tdR, tr, pdR, pr), c="orange", label ="P-R")
    ax3.grid(True)
    ax3.legend(loc='best', prop={'size': 8})
    ax3.set_ylabel("Difference in Dipole")
    ax3.set_xlabel('$\omega$')
    ax4.plot(wc, aRw(tdR, tr, EnR, er, wc), c="red", label="R")
    ax4.plot(wc, aTw(tdT, tt, EnT, et, wc), c="blue", label="TS")
    ax4.plot(wc, aPw(tdP, tp, EnP, ep, wc), c="orange", label="P")
    ax4.set_ylabel('Polarizability')
    ax4.grid(True)
    ax4.legend(loc='best', prop={'size': 8})
    ax4.set_xlabel('$\omega$')
    ax5.plot(wc, wc*(aTw(tdT, tt, EnT, et, wc) - aRw(tdR, tr, EnR, er, wc)), c="purple", label="TS-R")
    ax5.plot(wc, wc*(aPw(tdP, tp, EnP, ep, wc) - aRw(tdR, tr, EnR, er, wc)), c="orange", label="P-R")
    ax5.set_ylabel('w*(Difference in Polarizabilities)')
    ax5.grid(True)
    ax5.legend(loc='best', prop={'size': 8})
    ax5.set_xlabel('$\omega$')
    ax6.axhline(dT2(tdT, tt, pdT, pt) - dR2(tdR, tr, pdR, pr), c="purple", linestyle="--", label ="d^2 TS-R")
    ax6.axhline(dP2(tdP, tp, pdP, pp) - dR2(tdR, tr, pdR, pr), c="orange", linestyle="--", label ="d^2 P-R")
    ax6.plot(wc, wc*(aTw(tdT, tt, EnT, et, wc) - aRw(tdR, tr, EnR, er, wc)), c="purple", label="a TS-R")
    ax6.plot(wc, wc*(aPw(tdP, tp, EnP, ep, wc) - aRw(tdR, tr, EnR, er, wc)), c="orange", label="a P-R")
    ax6.set_ylabel('Dipole or Polarizability')
    ax6.grid(True)
    ax6.legend(loc='best', prop={'size': 8})
    ax6.set_xlabel('$\omega$')
    plt.show()


<>:6: SyntaxWarning: invalid escape sequence '\o'
<>:8: SyntaxWarning: invalid escape sequence '\o'
<>:17: SyntaxWarning: invalid escape sequence '\o'
<>:23: SyntaxWarning: invalid escape sequence '\o'
<>:30: SyntaxWarning: invalid escape sequence '\o'
<>:36: SyntaxWarning: invalid escape sequence '\o'
<>:44: SyntaxWarning: invalid escape sequence '\o'
<>:6: SyntaxWarning: invalid escape sequence '\o'
<>:8: SyntaxWarning: invalid escape sequence '\o'
<>:17: SyntaxWarning: invalid escape sequence '\o'
<>:23: SyntaxWarning: invalid escape sequence '\o'
<>:30: SyntaxWarning: invalid escape sequence '\o'
<>:36: SyntaxWarning: invalid escape sequence '\o'
<>:44: SyntaxWarning: invalid escape sequence '\o'
/state/partition1/job-59770034/ipykernel_2963219/2798060598.py:6: SyntaxWarning: invalid escape sequence '\o'
  ax1.set_title(f'm over $\omega$')
/state/partition1/job-59770034/ipykernel_2963219/2798060598.py:8: SyntaxWarning: invalid escape sequence '\o'
  ax1.set_xlabel('$\omega$')
/stat

In [17]:
interact(m, pr=(-10, 10, 0.2), pt=(-10, 10, 0.2), pp=(-10, 10, 0.2), 
         tr=(-0.3, 0.3, 0.001), tt=(-0.3, 0.3, 0.001), tp=(-0.3, 0.3, 0.001),
         er=(0, 3, 0.1), et=(0, 3, 0.1), ep=(0, 3, 0.1));


interactive(children=(FloatSlider(value=0.0, description='tr', max=0.3, min=-0.3, step=0.001), FloatSlider(val…